In [1]:
from nilmtk import DataSet
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py
import datetime
import warnings
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot

import itertools as it


#TODO create a folder with utils and split modules there

#print module
#file module
#setup module
#process module 
from utils import print_parameters, print_log
from utils import create_file, create_hdf5_group, store_single_hdf5, store_many_hdf5, read_many_hdf5
from utils import get_appliances, get_good_sections, get_clean_data, trasfrom_ts, append_images

plt.style.use('ggplot')
rcParams['figure.figsize'] = (12, 10)
plt.figure(figsize=(10,30))

dataset_name = "iawe"
dataset = DataSet('datasets/'+dataset_name+'.h5')
dataset.set_window()


Bad key "text.kerning_factor" on line 4 in
/Users/jakob/anaconda3/envs/nilmtk-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/jakob/anaconda3/envs/nilmtk-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1567, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 959, in H5FD_sec2_lock
    unable to lock file, errno = 35, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file 'datasets/iawe.h5'

# TIMESERIES TO IMAGE CONVERTER


In [4]:
par = {
'step_in_mins': 13, 
 'max_images': 1000, # number of images per appliance per building
 'img_size': 100, # output image size
 'frames': 5, # video frames
 'allowed_delta_between_frames': 600, # allowed time difference between frames
 'sample_period': 6, # can be obtained from dataset.metadata["sample_period"] but it is inconsistent
 'percentage_of_missing_data_allowed': 0.95, # missing data will be backfilled
 'add_brightness': True, # multiply non-zero mean power to image
 'ts_save': False, # save source time series 
 'trs_type': 'GAF', #GAF or RECU - Gramian Angural Field or Recurrance plot
 'trs_type_gaf': 'GASF', # GASF or GADF
 'multiple_buildings': True, 
 'selected_building': 1, # used is parameter above is False
 'manauly_select_appliances': False # can be set in get_appliances() function
}

In [5]:
#initialize 
param_setup(dataset,par)
#maybe return appliances #TODO 
get_appliances(dataset,par)
print_parameters(par)
file_name = create_file(par)

# define global metrics
healthy_appliances = set()
images_stacked = 0 
images_stacked_per_appliance = 0 

# collect at least N images of "max_dataset_size" for each applaiance for every building. #TODO 
for appliance in par["appliances"]:
    print_begin_appliance(appliance,par)

    # define metric
    images_stacked_per_appliance = 0
    
    # loop through all buildings 
    for building in dataset.buildings:
        print_begin_building(building,par)

        # define temporary array to store image / frame 
        img_stack_tmp = np.zeros([0, par["img_size"], par["img_size"]])
        sig_stack_tmp = np.zeros([0, par["ts_size"]])
        
        # define main array to store video
        img_stack = np.zeros([0, par["frames"], par["img_size"], par["img_size"]])
        sig_stack = np.zeros([0, par["frames"], par["ts_size"]])

        # case if using only selected building
        if par["multiple_buildings"] == False:
                if int(building)  != par["selected_building"]:
                    print("skipping building "f"{building}"" due to parameter muliple_buildings ")
                    continue
        
        # filter out appliances with appliance
        for meter in dataset.buildings[building].elec.submeters().meters:  
            
            # get appliance name 
            label = meter.appliances[0].metadata.get("type")
            
            # continue only for applaiance from first loop
            if label != appliance : continue 

            # load data into RAM
            signal,time_stamps = mount_data(meter)

            # slice timestamps and signal data to specified length
            time_stamps_slices = moving_window(time_stamps, par["ts_size"])
            signal_slices = moving_window(signal, par["ts_size"])

            # filter out data that is not needed
            signal_slices, time_stamps_slices = filter_empty_slices_and_fill_missing_samples(signal_slices, time_stamps_slices, par)
            signal_slices, time_stamps_slices = filter_low_entropy_slices(signal_slices, time_stamps_slices, print_parameters)
            
            # continue if no data
            if signal_slices.shape[0] == 0: continue
        
            print("Finished pre-processing! transforming...")

            #define metrics 
            last_stamp = 0 # used in append_images for calculating delta
            print_flag = 10 # variable helps reduce log output in print_progress 

            # trasform pre-processed signal slices 
            for i, [sig, time_stamps] in enumerate(zip(signal_slices, time_stamps_slices)):
                
                print_flag = print_progress(i, signal_slices, img_stack, print_flag, par)

                # stop looping if enough data
                if img_stack.shape[0] >= (par["max_images"]):
                    print_break(par)
                    break
                
                sig, img = trasfrom_ts(sig, par)
                    
                #append trasformed ts
                img_stack, img_stack_tmp, sig_stack, sig_stack_tmp, last_stamp = append_images(img, img_stack, img_stack_tmp,
                                                                                               sig, sig_stack, sig_stack_tmp,
                                                                                               time_stamps, last_stamp, par) 
        
        if img_stack.shape[0] > 0:
            # save images to hdf5
            group_path = f"{par['dataset_name']}/"f"{appliance}/"f"{building}"
        
            store_many_hdf5(file_name, img_stack[...,np.newaxis], group_path, "img", force_del="yes")
            
            images_stacked_per_appliance += img_stack.shape[0]
            images_stacked += img_stack.shape[0]
            healthy_appliances.add(appliance)
            
            if par["ts_save"]:
                #save source time series 
                store_many_hdf5(file_name, sig_stack, group_path, "sig", force_del="yes")

        else:
            print("empty for building", building, "appliance", appliance)
  
        print("finished building N", building)
        print_log(par, "finished building N", building)
    
    print_end_of_loop(images_stacked_per_appliance, appliance, par)
    
print_end(images_stacked, healthy_appliances, par)

file exists!


 Starting clothes iron (1/9):
Starting building  1
Finished pre-processing! transforming...
empty for building 1 appliance clothes iron
finished building N 1

number of images (per appliance) stacked: 0
finished clothes iron




 Starting wet appliance (2/9):
Starting building  1
Finished pre-processing! transforming...
procesed: 10.34% finished: 0.1%
procesed: 20.26% finished: 0.1%
procesed: 30.17% finished: 0.1%
procesed: 40.09% finished: 0.1%
procesed: 50.43% finished: 0.1%
procesed: 60.34% finished: 0.1%
procesed: 70.26% finished: 0.1%
procesed: 80.17% finished: 0.1%
procesed: 90.09% finished: 0.1%
storing... samples to store: 1
finshed. stored to iawe_GASF_13m_100S5X_9A1000N_AB_BRIG/iawe/wet appliance/1/img
finished building N 1

number of images (per appliance) stacked: 1
finished wet appliance




 Starting air conditioner (3/9):
Starting building  1
Finished pre-processing! transforming...
procesed: 10.18% finished: 0.9%
procesed: 20.16% finished: 1.5%
procesed: 